# Download data from Climate Data Store (CDS)

CDS provides free-to-use historical data and forecasts about EU climate variables.

## Getting started

1. Self-register to the CDS website https://cds.climate.copernicus.eu/cdsapp#!/home
2. Follow these instructions to set up the API client: https://cds.climate.copernicus.eu/api-how-to#use-the-cds-api-client-for-data-access
    1. Create a file .cdsapirc in your home directory
    2. Copy `url` and `key` in the .cdsapirc file
    3. Install the cdsapi Python library `pip install cdsapi` (make sure to select the right conda environment)

In [1]:
import cdsapi
c = cdsapi.Client()
output_folder = "./../../../s3/data/d003_climate/"

## Download raw data

Adapt the `retrieve` statement to download the desired data. You can also use the CDS Form to craft the request and then copy&paste the request.
You can also review the status of your requests in your profile page in the CDS website.

In [ ]:
c.retrieve(
    'reanalysis-cerra-single-levels',
    {
        'format': 'netcdf',
        'day': '31',
        'month': '08',
        'year': '1984',
        'product_type': 'analysis',
        'data_type': 'ensemble_members',
        'level_type': 'surface_or_atmosphere',
        'variable': '10m_wind_direction',
        'time': '00:00',
    },
    output_folder+'download.nc')

## Send processing workflow and download processed data
It is also possible to process data on the CDS server side and download the result. To do so, one has to write the processing routine in Python in a separate file (here called `CDS_API_workflow.py`) and send a `workflow` request along with the input parameters. After the processing has terminated, the resulting data can be downloaded as .nc file.

In [ ]:
# get city coordinates
lonlat_list =[["NL005C", 4.640960, 52.113299], ["NL006C", 5.384670, 52.173656], ["NL007C", 5.921886, 52.189884]]

In [ ]:
# open the script where the processing function is defined
with open("CDS_API_workflow.py") as f:
    code = f.read()
res = c.workflow(code, 
                 lonlat=lonlat_list, day_or_night="night", #keyword arguments passed to the function
                 workflow_name="get_daily_utci_stats") # workflow_name must be equal to the name of the function defined in the script
c.download(res, targets=[output_folder + "output.nc"])

## Read the downloaded .nc file with xarray

In [ ]:
import xarray as xr
climate_path = output_folder + "output.nc"
ds_disk = xr.open_dataset(climate_path)
print(ds_disk)

In [ ]:
print(ds_disk.coords)

In [ ]:
print(ds_disk.dims)

In [ ]:
ds_disk.sel(statistic="max", city_lonlat="NL005C").utci.plot()

## Convert to dataframe

In [ ]:
df = ds_disk.to_dataframe()["utci"]
df